In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

In [8]:
import tensorflow as tf

In [3]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [5]:
IMAGE_SIZE = [128, 128]

In [6]:
from tensorflow.keras.applications.xception import decode_predictions

In [33]:
xception=tf.keras.applications.xception.Xception(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [34]:
for layer in xception.layers:
  layer.trainable = False

In [35]:
x = Flatten()(xception.output)
prediction = Dense(4, activation='softmax')(x)

In [36]:
prediction.shape

(None, 4)

In [37]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 63, 63,    │        864 │ input_layer_2[0]… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 63, 63,    │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 63, 63,    │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 61, 61,    │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 61, 61,    │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 61, 61,    │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 61, 61,    │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 61, 61,    │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 61, 61,    │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 61, 61,    │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 61, 61,    │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 31, 31,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 31, 31,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 31, 31,    │        512 │ conv2d_8[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_24 (Add)        │ (None, 31, 31,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 31, 31,    │          0 │ add_24[0][0]    

 Total params: 20,992,556 (80.08 MB)

 Trainable params: 131,076 (512.02 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [38]:
import os
import cv2

In [21]:
def load_images(directory_path, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory_path, filename)
            img = load_image(img_path, target_size=target_size)  
            images.append(img)
    return np.array(images)



def load_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    img = cv2.resize(img, target_size)
    return img



In [22]:
covid_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//COVID//images'
opacity_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Lung_Opacity//images'
normal_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Normal//images'
pneumonia_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Viral Pneumonia//images'


covid_images = load_images(covid_images_dir)
opacity_images = load_images(opacity_images_dir)
normal_images = load_images(normal_images_dir)
pneumonia_images = load_images(pneumonia_images_dir)


X = np.concatenate([covid_images, opacity_images, normal_images, pneumonia_images], axis=0)

In [23]:
covid_label = 0
opacity_label = 1
normal_label = 2
pneumonia_label = 3

covid_labels = np.full((len(covid_images),), covid_label)
opacity_labels = np.full((len(opacity_images),), opacity_label)
normal_labels = np.full((len(normal_images),), normal_label)
pneumonia_labels = np.full((len(pneumonia_images),), pneumonia_label)

In [24]:
Y = np.concatenate([covid_labels, opacity_labels, normal_labels, pneumonia_labels], axis=0)


In [25]:
shuffle_index = np.random.permutation(len(X))
X_ = X[shuffle_index]
Y_ = Y[shuffle_index]

In [26]:
from keras.utils import to_categorical

# Assuming Y_train is loaded or generated with integer labels
Y_ = to_categorical(Y_, num_classes=4)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.2, random_state=42)

In [39]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(f"Y_train shape before: {Y_train.shape}") 
history = model.fit(X_train, Y_train, batch_size=16, epochs=10, validation_data=(X_test, Y_test))
train_loss, train_accuracy = model.evaluate(X_train, Y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Y_train shape before: (16932, 4)
Epoch 1/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 557s 521ms/step - accuracy: 0.6104 - loss: 36.9522 - val_accuracy: 0.7087 - val_loss: 17.6644
Epoch 2/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 623s 588ms/step - accuracy: 0.6995 - loss: 22.2377 - val_accuracy: 0.6664 - val_loss: 47.7745
Epoch 3/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 626s 591ms/step - accuracy: 0.7280 - loss: 23.2815 - val_accuracy: 0.7817 - val_loss: 22.7963
Epoch 4/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 593s 560ms/step - accuracy: 0.7500 - loss: 21.8385 - val_accuracy: 0.6364 - val_loss: 52.6362
Epoch 5/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 520s 491ms/step - accuracy: 0.7482 - loss: 23.6091 - val_accuracy: 0.7090 - val_loss: 28.1522
Epoch 6/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 508s 480ms/step - accuracy: 0.7466 - loss: 24.7174 - val_accuracy: 0.7470 - val_loss: 22.3154
Epoch 7/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 542s 512ms/step - accuracy: 0.7715 - loss: 19.8593 - val_accuracy: 0.7879 - val_loss: 19.0901
Epoch 8/10
1059

KeyboardInterrupt: 